In [1]:
SAT_FN = 'SAT_Results.csv'
HSD_FN = 'DOE_High_School_Directory_2014-2015.csv'

In [2]:
import csv
with open(HSD_FN, 'rb') as fi:
    reader = csv.reader(fi)
    print reader.next()
    print reader.next()
#     print fi.next()
#     print fi.next()
#     print fi.next()
#     print fi.next()

['dbn', 'school_name', 'boro', 'building_code', 'phone_number', 'fax_number', 'grade_span_min', 'grade_span_max', 'expgrade_span_min', 'expgrade_span_max', 'bus', 'subway', 'primary_address_line_1', 'city', 'state_code', 'zip', 'website', 'total_students', 'campus_name', 'school_type', 'overview_paragraph', 'program_highlights', 'language_classes', 'advancedplacement_courses', 'online_ap_courses', 'online_language_courses', 'extracurricular_activities', 'psal_sports_boys', 'psal_sports_girls', 'psal_sports_coed', 'school_sports', 'partner_cbo', 'partner_hospital', 'partner_highered', 'partner_cultural', 'partner_nonprofit', 'partner_corporate', 'partner_financial', 'partner_other', 'addtl_info1', 'addtl_info2', 'start_time', 'end_time', 'se_services', 'ell_programs', 'school_accessibility_description', 'number_programs', 'priority01', 'priority02', 'priority03', 'priority04', 'priority05', 'priority06', 'priority07', 'priority08', 'priority09', 'priority10', 'Location 1']
['01M292', 'H

In [3]:
df = spark.read \
        .format('com.databricks.spark.csv') \
        .option('header','true') \
        .option('inferSchema','true') \
        .option('parserLib','UNIVOCITY') \
        .load(HSD_FN)

In [4]:
sat = sc.textFile(SAT_FN,use_unicode=False).cache()

In [5]:
list(enumerate(sat.first().split(',')))

[(0, 'DBN'),
 (1, 'SCHOOL NAME'),
 (2, 'Num of SAT Test Takers'),
 (3, 'SAT Critical Reading Avg. Score'),
 (4, 'SAT Math Avg. Score'),
 (5, 'SAT Writing Avg. Score')]

In [6]:
noHeaderSat = sat.filter(lambda x: not x.startswith('DBN,'))
noHeaderSat.first()

'02M047,47 THE AMERICAN SIGN LANGUAGE AND ENGLISH SECONDARY SCHOOL,16,395,400,387'

In [7]:
def extractScores(partitionId,partition):
    if partitionId==0:
        partition.next()
    import csv
    reader = csv.reader(partition)
    for row in reader:
        if row[2] !='s':
            (dbn,takers,score) = (row[0], int(row[2]), int(row[4]))
            yield (dbn, (score*takers, takers))

satScores = sat.mapPartitionsWithIndex(extractScores)
satScores.take(5)

[('02M047', (6400, 16)),
 ('21K410', (207575, 475)),
 ('30Q301', (43120, 98)),
 ('17K382', (22066, 59)),
 ('18K637', (13335, 35))]

In [8]:
sat.getNumPartitions()

2

In [9]:
schools = sc.textFile(HSD_FN,use_unicode=False).cache()
list(enumerate(schools.first().split(',')))

[(0, 'dbn'),
 (1, 'school_name'),
 (2, 'boro'),
 (3, 'building_code'),
 (4, 'phone_number'),
 (5, 'fax_number'),
 (6, 'grade_span_min'),
 (7, 'grade_span_max'),
 (8, 'expgrade_span_min'),
 (9, 'expgrade_span_max'),
 (10, 'bus'),
 (11, 'subway'),
 (12, 'primary_address_line_1'),
 (13, 'city'),
 (14, 'state_code'),
 (15, 'zip'),
 (16, 'website'),
 (17, 'total_students'),
 (18, 'campus_name'),
 (19, 'school_type'),
 (20, 'overview_paragraph'),
 (21, 'program_highlights'),
 (22, 'language_classes'),
 (23, 'advancedplacement_courses'),
 (24, 'online_ap_courses'),
 (25, 'online_language_courses'),
 (26, 'extracurricular_activities'),
 (27, 'psal_sports_boys'),
 (28, 'psal_sports_girls'),
 (29, 'psal_sports_coed'),
 (30, 'school_sports'),
 (31, 'partner_cbo'),
 (32, 'partner_hospital'),
 (33, 'partner_highered'),
 (34, 'partner_cultural'),
 (35, 'partner_nonprofit'),
 (36, 'partner_corporate'),
 (37, 'partner_financial'),
 (38, 'partner_other'),
 (39, 'addtl_info1'),
 (40, 'addtl_info2'),
 (4

In [10]:
def extractSchools(partitionId, partition):
    if partitionId==0:
        partition.next()
    import csv
    reader = csv.reader(partition)
    for row in reader:
        if len(row)==58 and row[17].isdigit():
            (dbn,boro,total_students) = (row[0],row[2], int(row[17]))
            if total_students>500:
                yield (dbn,boro)
    
largeSchools = schools.mapPartitionsWithIndex(extractSchools)
largeSchools.take(5)

[('01M450', 'Manhattan'),
 ('01M539', 'Manhattan'),
 ('01M696', 'Manhattan'),
 ('02M374', 'Manhattan'),
 ('02M400', 'Manhattan')]

In [11]:
largeSchools.join(satScores).values().take(5)

[('Bronx', (23069, 59)),
 ('Staten Island', (52216, 107)),
 ('Bronx', (16317, 49)),
 ('Brooklyn', (33235, 85)),
 ('Bronx', (24570, 65))]

In [12]:
largeSchools.join(satScores).values() \
    .reduceByKey(lambda x,y: (x[0]+y[0],x[1]+y[1])) \
    .mapValues(lambda x: x[0]/x[1]) \
    .take(5)

[('Bronx', 470),
 ('Manhattan', 514),
 ('Brooklyn', 487),
 ('Staten Island', 477),
 ('Queens', 474)]

In [13]:
bus = sc.textFile(HSD_FN,use_unicode=False).cache()

In [38]:
def extractBus(partitionId, partition):
    if partitionId==0:
        partition.next()
    import csv
    reader = csv.reader(partition)
    for row in reader:
        if len(row)==58 and row[17].isdigit():
            (dbn ,bus) = (row[0],row[10])
            for busLines in bus.split(', '):
                yield (dbn, busLines)
    
bus_data = schools.mapPartitionsWithIndex(extractBus)

def extractSubway(partitionId, partition):
    if partitionId==0:
        partition.next()
    import csv
    reader = csv.reader(partition)
    for row in reader:
        if len(row)==58 and row[17].isdigit():
            (dbn, subway) = (row[0],row[11])
            for subwayLines in subway.split(', '):
                yield (dbn, subwayLines)
    
subway_data = schools.mapPartitionsWithIndex(extractSubway)
#bus_data.take(5)

In [49]:
a = bus_data.join(satScores).values() \
            .reduceByKey(lambda x,y: (x[0]+y[0],x[1]+y[1])) \
            .mapValues(lambda x: x[0]/x[1])
b = subway_data.join(satScores).values() \
                .reduceByKey(lambda x,y: (x[0]+y[0],x[1]+y[1])) \
                .mapValues(lambda x: x[0]/x[1])

In [52]:
final = a.collect() + b.collect()
sorted(final)

[('1', 532),
 ('1 to 137th St - City College ; 2', 399),
 ('1 to 137th St - City College ; A', 516),
 ('1 to 157th St ; B', 376),
 ('1 to 18th St ; 2', 498),
 ('1 to 191st St ; A to 190th St', 384),
 ('1 to 231st St', 393),
 ('1 to 66th St - Lincoln Center ; 2', 514),
 ('1 to 66th St - Lincoln Center ; A', 575),
 ('1 to Chambers St ; 2', 440),
 ('1 to Dyckman St ; A to 190th St', 446),
 ('1 to Marble Hill - 225th St', 406),
 ('1 to Rector St ; 2', 489),
 ('2', 494),
 ('2 to 149th St-Grand Concourse ; 3 to 145th St ; 4', 365),
 ('2 to 149th St-Grand Concourse ; 4', 381),
 ('3', 547),
 ('3 to 34th St - Penn Station ; A', 423),
 ('3 to 72nd St ; A', 514),
 ('3 to Central Park North-110th St ; 6 to 103rd St', 446),
 ('3 to Central Park North-110th St ; 6 to 103rd St ; 6 to 110th St', 421),
 ('3 to Eastern Parkway-Brooklyn Museum ; C to Franklin Ave ; F', 364),
 ('3 to Harlem - 148 St ; A', 474),
 ('3 to Hoyt St ; A', 402),
 ('3 to Pennsylvania Ave ; C to Van Siclen Ave ; L to Livonia Ave',